## Web scraping automation with Selenium. Forebet.com predictions U_O 2.5 

#### Import libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


KeyboardInterrupt



#### Activate ChromeDriver and open a new session of Chrome

In [ ]:
# Specify the path to the ChromeDriver executable
chromedriver_path = 'C:\WebDrivers\chromedriver.exe'

# Create a new service object with the specified executable path
service = Service(chromedriver_path)

# Configure the ChromeDriver options
options = Options()
options.binary_location = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
#options.add_argument('--headless') # Run Chrome in headless mode

# Create a new Chrome WebDriver instance with the custom service and options
driver = webdriver.Chrome(service=service, options=options)

#### Open the URL

In [ ]:
# driver get the URL
driver.get('https://www.forebet.com/en/football-predictions/under-over-25-goals')

#### Press the "Consent" button. Use one of the option below 

In [ ]:
# 1st way. Wait for the cookie button to be clickable and press consent
#WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.fc-button.fc-cta-consent.fc-primary-button > p'))).click()

In [ ]:
# 2nd way. Wait for the cookie button to be clickable and press consent
consent_button = driver.find_element("xpath", '(//p[@class="fc-button-label"])[1]')
consent_button.click()

#### Find a way to overpass the iframe. Work in progress....
For the moment this step should be done by manually click the iframe arrow when the pop up appears in the bottom down 

In [ ]:
...

#### Find and click "More" button to load all the games

In [ ]:
# Find and click the "More" button
for i in range(50):
    try:
        # Wait for the element to become visible and clickable
        element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#mrows span")))
        # Click the element
        element.click()
    except:
        print("No 'more' buttons to click!")
        break

#### Define the data frame columns

In [ ]:
# define data frame
df_pred = pd.DataFrame(columns=['Date Time', 'Home Team', 'Away Team', 'Under Prob',
                                'Over Prob', 'U/O', 'Predicted Score', 'Odds'])

#### Extract the data and compare the lenght for each column

In [ ]:
date = driver.find_elements("xpath", '//time[@itemprop="startDate"]/span')

In [ ]:
len(date)

In [ ]:
home_team = driver.find_elements("xpath", '(//span[@class="homeTeam"])/span[1]')

In [ ]:
len(home_team)

In [ ]:
away_team = driver.find_elements("xpath", '(//span[@class="awayTeam"])/span[1]')

In [ ]:
len(away_team)

In [ ]:
under_prob = driver.find_elements("xpath", '(//div[@class="fprc"])/span[1]')

In [ ]:
len(under_prob)

In [ ]:
over_prob = driver.find_elements("xpath", '(//div[@class="fprc"])/span[2]')

In [ ]:
len(over_prob)

In [ ]:
u_o = driver.find_elements("xpath", '(//span[starts-with(@class, "forepr ")])/span')

In [ ]:
len(u_o)

In [ ]:
predicted_score = driver.find_elements("xpath", '(//div[starts-with(@class, "rcnt ")])/div[5]')

In [ ]:
len(predicted_score)

In [ ]:
odds = driver.find_elements("xpath", '(//div[starts-with(@class, "rcnt ")])/div[8]/span')

In [ ]:
len(odds)

In [ ]:
len(u_o)

#### Append all the data to the Data Frame and preview the df_pred
Here there is room for improvement. Right now it takes a bit of time to append due to the amount of data it needs to scrape.

In [ ]:
# in Dataframe appenden
for i in range(len(odds)):

    df_pred =df_pred.append({'Date Time':date[i].text, 'Home Team': home_team[i].text,
                            'Away Team': away_team[i].text, 'Under Prob': under_prob[i].text, 
                            'Over Prob': over_prob[i].text, 'U/O': u_o[i].text,
                            'Predicted Score': predicted_score[i].text, 'Odds': odds[i].text}, ignore_index=True)

In [ ]:
df_pred

#### Split the columns "Date", "Predicted Score" and "Probability" and drop them

In [ ]:
df_pred[['Date', 'Time']] = df_pred['Date Time'].str.split(' ', expand=True)
df_pred[['Score Home', 'Score Away']] = df_pred['Predicted Score'].str.split('-', expand=True)

# drop the columns not needed
df_pred = df_pred.drop(['Date Time', 'Predicted Score'], axis=1)

#### Re-arrange the columns

In [ ]:
# re arrange the columns 
df_pred = df_pred[['Date', 'Time', 'Home Team', 'Away Team', 'Under Prob', 'Over Prob', 'U/O', 'Score Home', 
         'Score Away', 'Odds']]

In [ ]:
df_pred

#### Store the data in CSV format 

In [ ]:
# create csv
df_pred.to_csv('Extract O-U 2.5.csv', encoding='utf-8', index=False)